In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (accuracy_score, recall_score, precision_score, 
                            f1_score, roc_auc_score, log_loss, confusion_matrix)
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay

import mlflow
import mlflow.sklearn

import os

# Criar diretório mlruns se não existir
if not os.path.exists("mlruns"):
    os.makedirs("mlruns")

In [ ]:
credito = pd.read_csv('Credit.csv')
credito.shape

In [ ]:
credito.head()

In [ ]:
for col in credito.columns:
    if credito[col].dtype == 'object':
        credito[col] = credito[col].astype('category').cat.codes

In [ ]:
credito.head()

In [ ]:
previsores = credito.iloc[:,0:20].values
classe = credito.iloc[:,20].values

In [ ]:
previsores

In [ ]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,classe,
                                        test_size=0.3,random_state=123)

In [ ]:
# Configurar o tracking URI para o diretório local
mlflow.set_tracking_uri("file:./mlruns")

mlflow.set_experiment("nbexperimento")

with mlflow.start_run():
    naive_bayes = GaussianNB()
    naive_bayes.fit(X_treinamento, y_treinamento)
    previsoes = naive_bayes.predict(X_teste)
    
    #métricas
    acuracia = accuracy_score(y_teste, previsoes)
    recall = recall_score(y_teste, previsoes)
    precision = precision_score(y_teste, previsoes)
    f1 = f1_score(y_teste, previsoes)
    auc= roc_auc_score(y_teste, previsoes)
    log_loss_value = log_loss(y_teste, previsoes)  # Renomeado para evitar conflito
    
    #registrar métricas
    mlflow.log_metric("acuracia",acuracia)
    mlflow.log_metric("recall",recall)
    mlflow.log_metric("precision",precision)
    mlflow.log_metric("f1",f1)
    mlflow.log_metric("auc",auc)
    mlflow.log_metric("log_loss",log_loss_value)
    
    #gráficos
    cm = confusion_matrix(y_teste, previsoes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.savefig("confusion.png")
    mlflow.log_artifact("confusion.png")
    plt.close()

    roc = RocCurveDisplay.from_estimator(naive_bayes, X_teste, y_teste)
    plt.savefig("roc.png")
    mlflow.log_artifact("roc.png")
    plt.close() 
    
    #modelo
    mlflow.sklearn.log_model(naive_bayes,"ModeloNB")
    
    #informações da execução
    print("Modelo: ", mlflow.active_run().info.run_id)